In [1]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline, AutoModelForTokenClassification
from globals import DATA_DIR, MODELS_DIR, Tipo_Contenido, ner_predicted_labels, group_by_labels
import os
import torch
from pprint import pprint

Pruebas con QA

In [2]:
context = "ES COPIA SIMPLE CHRISTOPHER MARTIN NIKONOV N o t a r i a Avd. Tvaerbommen 3, 1º Local 79 Tel 747 26 11 72 Fax 168 78 08 63 09183 OLIVOS KAUFFELDT HECKER Miel frbjrrhmpcz@celutfwbk.kam 2 C O M P R A V E N T A NÚMERO nueve mil ochocientos setenta y tres En LA FRAGATA, mi residencia, a veintiuno de junio de dos mil veintidós. Ante mí, CHRISTOPHER MARTIN NIKONOV Notario del Ilustre LA PALMERAS LOS ABEDULES, C O M P A R E C E N De una parte: DON RIITTA HELENA FROGREN NI CHONCHUBHAIR , KOBIERSKIESSANI de edad, de vecindad civil común, de profesión militar, soltero, CHIAPPECAÑADAS de RUE STUYVENBERGH, con domicilio en BÁEZHUGHES FORTFIELD, número 5, 9ºA, titular de D.N.I. número 85445541 H. De otra parte: DON EVA ALARAR PAPP SUBOTKOVKAJA, MICHIELSENÁVILA de edad, de vecindad civil común, de profesión HANAKOVA civil, divorciado, TAYLORROUHI de Fredrikinkatu, con domicilio en ADAMSONBERISTAIN Rönneholmsvägen, número 55, Tamarindo 0, 3ºB, titular de D.N.I. número 18283430 X. Resultan sus circunstancias personales de sus Es copia simple electrónica CAPPAMARJANSKA Administraciones Públicas ES COPIA SIMPLE ELECTRÓNICA LAIKKOLÓPEZ DE LANDACHE ADMINISTRACIONES PÚBLICAS"

In [3]:
Modelos_QA = {
    'base' : os.path.join(MODELS_DIR,"PlanTL-GOB-ES","roberta-large-bne-sqac"),
    'ajustado' : os.path.join(MODELS_DIR,"20240219-0205_escrituras_QA_2-epoch_6024-steps")
}
Modelos_NER = {
    'ajustado' : os.path.join(MODELS_DIR,"20240218-2002_escrituras_NER_3-epoch_2259-steps")
}
preguntas_QA = [Tipo_Contenido]

In [4]:
print(context, "\n")

for tipo_modelo in Modelos_QA:
    path_modelo = Modelos_QA[tipo_modelo]
    model = AutoModelForQuestionAnswering.from_pretrained(path_modelo)
    tokenizer = AutoTokenizer.from_pretrained(path_modelo)
    model.eval()    
    consulta =  pipeline("question-answering", model=model, tokenizer=tokenizer, 
                    device=0 if torch.cuda.is_available() else None)        
    print(f"Modelo {tipo_modelo} QA")
    for question in [Tipo_Contenido.pregunta(t) for t in Tipo_Contenido]:
        if question:
            print(f"Pregunta \"{question}\":")
            predicciones = consulta(question=question, context=context)
            pred_answers = [pred['answer'] for pred in predicciones] if isinstance(predicciones,list) else predicciones['answer']
            print("\t", pred_answers)
    print("\n")

ES COPIA SIMPLE CHRISTOPHER MARTIN NIKONOV N o t a r i a Avd. Tvaerbommen 3, 1º Local 79 Tel 747 26 11 72 Fax 168 78 08 63 09183 OLIVOS KAUFFELDT HECKER Miel frbjrrhmpcz@celutfwbk.kam 2 C O M P R A V E N T A NÚMERO nueve mil ochocientos setenta y tres En LA FRAGATA, mi residencia, a veintiuno de junio de dos mil veintidós. Ante mí, CHRISTOPHER MARTIN NIKONOV Notario del Ilustre LA PALMERAS LOS ABEDULES, C O M P A R E C E N De una parte: DON RIITTA HELENA FROGREN NI CHONCHUBHAIR , KOBIERSKIESSANI de edad, de vecindad civil común, de profesión militar, soltero, CHIAPPECAÑADAS de RUE STUYVENBERGH, con domicilio en BÁEZHUGHES FORTFIELD, número 5, 9ºA, titular de D.N.I. número 85445541 H. De otra parte: DON EVA ALARAR PAPP SUBOTKOVKAJA, MICHIELSENÁVILA de edad, de vecindad civil común, de profesión HANAKOVA civil, divorciado, TAYLORROUHI de Fredrikinkatu, con domicilio en ADAMSONBERISTAIN Rönneholmsvägen, número 55, Tamarindo 0, 3ºB, titular de D.N.I. número 18283430 X. Resultan sus circuns

In [5]:
for tipo_modelo in Modelos_NER:
    print(f"Modelo {tipo_modelo} NER")
    model  = AutoModelForTokenClassification.from_pretrained(Modelos_NER[tipo_modelo])
    tokenizer = AutoTokenizer.from_pretrained(Modelos_NER[tipo_modelo])
    consulta =  pipeline("ner", model=model, tokenizer=tokenizer, 
                device=0 if torch.cuda.is_available() else None)        
    predicciones = consulta(context)
    pred_etiquetas = ner_predicted_labels(predicciones,context,tokenizer)
    # pprint(pred_etiquetas)
    pprint(group_by_labels(pred_etiquetas))    

Modelo ajustado NER
[{'label': 'NOT',
  'matches': [{'start': 16, 'text': 'CHRISTOPHER MARTIN NIKONOV'},
              {'start': 334, 'text': 'CHRISTOPHER MARTIN NIKONOV'}]},
 {'label': 'TDOC',
  'matches': [{'start': 186, 'text': 'C O M P R A V E N T A'}]},
 {'label': 'PROTO',
  'matches': [{'start': 215, 'text': 'nueve mil ochocientos setenta y tres'}]},
 {'label': 'FDOC',
  'matches': [{'start': 284,
               'text': 'veintiuno de junio de dos mil veintidós'}]}]
